In [1]:
import json
import requests
from xml.etree import ElementTree

url = "http://localhost:9003/?output_format=jsonl"

text = "Na płocie siedzi kot."

In [2]:
r = requests.post(url, data = text.encode('utf-8'))

In [7]:
def parse_line(line):
    r = requests.post(url, data = line.encode('utf-8'))
    out = json.loads(r.text)
    return out[0]

In [3]:
r.headers

{'Server': 'gunicorn/19.9.0', 'Date': 'Tue, 20 Oct 2020 16:50:55 GMT', 'Connection': 'close', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '173'}

In [4]:
r.text

'[[["Na", "na", "prep:loc"], ["płocie", "płot", "subst:sg:loc:m3"], ["siedzi", "siedzieć", "fin:sg:ter:imperf"], ["kot", "kot", "subst:sg:nom:m2"], [".", ".", "interp"]]]\n'

In [5]:
out = json.loads(r.text)

In [6]:
out[0]

[['Na', 'na', 'prep:loc'],
 ['płocie', 'płot', 'subst:sg:loc:m3'],
 ['siedzi', 'siedzieć', 'fin:sg:ter:imperf'],
 ['kot', 'kot', 'subst:sg:nom:m2'],
 ['.', '.', 'interp']]

In [10]:
with open('train-raw.txt') as f:
    lines = [line.rstrip() for line in f]
tree = ElementTree.parse('train-gold.xml')
root = tree.getroot()

In [11]:
print(parse_line(lines[0]))

[['A', 'a', 'conj'], ['mogę', 'móc', 'fin:sg:pri:imperf'], [',', ',', 'interp'], ['bo', 'bo', 'comp'], ['moim', 'mój', 'adj:sg:inst:n:pos'], ['zdaniem', 'zdanie', 'subst:sg:inst:n'], ['jest', 'być', 'fin:sg:ter:imperf'], ['do', 'do', 'prep:gen'], ['niczego', 'nic', 'subst:sg:gen:n'], ['.', '.', 'interp']]


In [13]:
chunks = tree.find("chunkList").findall("chunk")

for line in chunks:
    for sentence in line.findall("chunk"):
        for word in sentence.findall("tok"):
            orth = word.find("orth").text
            base = word.find("lex").find("base").text
            ctag = word.find("lex").find("ctag").text

            print(f"{orth} {base} {ctag}")

A a conj
mogę móc fin:sg:pri:imperf
, , interp
bo bo comp
moim mój adj:sg:inst:n:pos
zdaniem zdanie subst:sg:inst:n
jest być fin:sg:ter:imperf
do do prep:gen
niczego nic subst:sg:gen:n
. . interp
I i qub
to to qub
delikatnie delikatnie adv:pos
mówiąc mówić pcon:imperf
. . interp
. . interp
. . interp
Rzecz rzecz subst:sg:nom:f
gustu gust subst:sg:gen:m3
:) :) subst:sg:nom:n
Ja ja ppron12:sg:nom:f:pri
sobie siebie siebie:dat
z z prep:inst:nwok
przyjemnością przyjemność subst:sg:inst:f
obejrzała obejrzeć praet:sg:f:perf
m być aglt:sg:pri:imperf:nwok
wczoraj wczoraj adv
" " interp
Live Live subst:sg:acc:m3
and and subst:sg:acc:m3
Let Let subst:sg:acc:m3
Die Die subst:sg:acc:m3
" " interp
. . interp
Nareszcie nareszcie adv
bez bez prep:gen:nwok
Connery'ego Connery subst:sg:gen:m1
! ! interp
! ! interp
! ! interp
Pasuje pasować fin:sg:ter:imperf
. . interp
Najbardziej bardzo adv:sup
ze z prep:gen:wok
wszystkich wszystek adj:pl:gen:m1:pos
. . interp
Może może qub
trochę trochę adv
za za qub


. . interp
Życzymy życzyć fin:pl:pri:imperf
powodzenia powodzenie subst:sg:gen:n
! ! interp
S S brev:pun
. . interp
C C brev:pun
. . interp
Małgorzata Małgorzata subst:sg:nom:f
Pławecka Pławecka subst:sg:nom:f
, , interp
była były adj:sg:nom:f:pos
uczennica uczennica subst:sg:nom:f
SP SP subst:sg:gen:f
nr numer brev:npun
5 5 num:pl:nom:m3:rec
została zostać praet:sg:f:perf
powołana powołać ppas:sg:nom:f:perf:aff
do do prep:gen
Reprezentacji reprezentacja subst:sg:gen:f
Polski Polska subst:sg:gen:f
na na prep:acc
Mistrzostwa mistrzostwo subst:pl:acc:n
Europy Europa subst:sg:gen:f
. . interp
Pchnięcie pchnięcie subst:sg:nom:n
kulą kula subst:sg:inst:f
i i conj
skok skok subst:sg:nom:m3
wzwyż wzwyż adv
to to pred
jej on ppron3:sg:gen:f:ter:akc:npraep
domena domena subst:sg:nom:f
. . interp
Zawodniczka zawodniczka subst:sg:nom:f
pcha pchać fin:sg:ter:imperf
8,80 8,80 num:pl:acc:m3:rec
m metr brev:npun
, , interp
a a conj
rekord rekord subst:sg:nom:m3
świata świat subst:sg:gen:m3
wynosi wyn

Maciura Maciura subst:sg:nom:m1
. . interp
Już już qub
zdążyli zdążyć praet:pl:m1:perf
się się qub
rozłożyć rozłożyć inf:perf
, , interp
ich on ppron3:pl:gen:m1:ter:akc:npraep
wielkie wielki adj:pl:nom:n:pos
ciała ciało subst:pl:nom:n
zgniły zgnić praet:pl:n:perf
, , interp
trupi trupi adj:sg:nom:m3:pos
odór odór subst:sg:nom:m3
wypełnił wypełnić praet:sg:m3:perf
trumny trumna subst:pl:acc:f
, , interp
mięso mięso subst:sg:nom:n
odpadło odpaść praet:sg:n:perf
od od prep:gen:nwok
grubokościstych grubokościsty adj:pl:gen:m3:pos
szkieletów szkielet subst:pl:gen:m3
, , interp
mocnych mocny adj:pl:gen:f:pos
kości kość subst:pl:gen:f
udowych udowy adj:pl:gen:f:pos
, , interp
szerokich szeroki adj:pl:gen:f:pos
miednic miednica subst:pl:gen:f
, , interp
dużych duży adj:pl:gen:f:pos
czaszek czaszka subst:pl:gen:f
. . interp
Zdarza zdarzać fin:sg:ter:imperf
się się qub
, , interp
napisał napisać praet:sg:m1:perf
by by qub
w w prep:loc:nwok
tym ten adj:sg:loc:n:pos
miejscu miejsce subst:sg:loc:n


wiedziała wiedzieć praet:sg:f:imperf
, , interp
to to conj
Jan Jan subst:sg:nom:m1
Buracki Buracki subst:sg:nom:m1
jest być fin:sg:ter:imperf
prezesem prezes subst:sg:inst:m1
i i conj
współwłaścicielem współwłaściciel subst:sg:inst:m1
firmy firma subst:sg:gen:f
budowlanej budowlany adj:sg:gen:f:pos
Buracki Buracki subst:sg:nom:m1
i i conj
Ska spółka brev:npun
. . interp
Pewnie pewnie qub
ciekawi ciekawić fin:sg:ter:imperf
Cię ty ppron12:sg:acc:f:sec:nakc
, , interp
kim kto subst:sg:inst:m1
jest być fin:sg:ter:imperf
pani pani subst:sg:nom:f
Kownacka Kownacka subst:sg:nom:f
? ? interp
Otóż otóż qub
jest być fin:sg:ter:imperf
żoną żona subst:sg:inst:f
Marka Marek subst:sg:gen:m1
Kownackiego Kownacki subst:sg:gen:m1
, , interp
twojego twój adj:sg:gen:m1:pos
ulubionego ulubiony adj:sg:gen:m1:pos
pana pan subst:sg:gen:m1
" " interp
czyste czysty adj:pl:nom:f:pos
ręce ręka subst:pl:nom:f
" " interp
. . interp
Radny radny subst:sg:nom:m1
Kownacki Kownacki subst:sg:nom:m1
rozumnie rozumnie adv

was wy ppron12:pl:gen:m1:sec
, , interp
chociażby chociażby qub
bez bez prep:gen:nwok
fikcji fikcja subst:sg:gen:f
waszego wasz adj:sg:gen:n:pos
istnienia istnieć ger:sg:gen:n:imperf:aff
, , interp
nasze nasz adj:sg:nom:n:pos
mocarstwo mocarstwo subst:sg:nom:n
traci tracić fin:sg:ter:imperf
tożsamość tożsamość subst:sg:acc:f
, , interp
dotychczasową dotychczasowy adj:sg:acc:f:pos
formę forma subst:sg:acc:f
, , interp
w w prep:loc:nwok
pewnym pewien adj:sg:loc:m3:pos
sensie sens subst:sg:loc:m3
wszystko wszystko subst:sg:acc:n
. . interp
Cóż cóż subst:sg:acc:n
więc więc conj
mogę móc fin:sg:pri:imperf
powiedzieć powiedzieć inf:perf
o o prep:loc
epoce epoka subst:sg:loc:f
, , interp
w w prep:loc:nwok
jakiej jaki adj:sg:loc:f:pos
przypadło przypaść praet:sg:n:perf
mi ja ppron12:sg:dat:m1:pri:nakc
żyć żyć inf:imperf
? ? interp
Jak jak adv:pos
widzę widzieć fin:sg:pri:imperf
procesy proces subst:pl:acc:m3
zachodzące zachodzić pact:pl:acc:m3:imperf:aff
w w prep:loc:nwok
sztuce sztuka subst:s

...

STRUKTURAMI struktura subst:pl:inst:f
, , interp
BARDZIEJ bardzo adv:com
CHCĘ chcieć fin:sg:pri:imperf
DOJŚĆ dojść inf:perf
DO do prep:gen
KONKRETNEJ konkretny adj:sg:gen:f:pos
WYPOWIEDZI wypowiedź subst:sg:gen:f
. . interp
NIE nie qub
PRZESZKADZA przeszkadzać fin:sg:ter:imperf
MI ja ppron12:sg:dat:m1:pri:nakc
MIESZANIE mieszać ger:sg:nom:n:imperf:aff
, , interp
WYRZUCANIE wyrzucać ger:sg:nom:n:imperf:aff
PEWNYCH pewien adj:pl:gen:f:pos
SCEN scena subst:pl:gen:f
, , interp
DODAWANIE dodawać ger:sg:nom:n:imperf:aff
CZEGOŚ coś subst:sg:gen:n
INNEGO inny adj:sg:gen:n:pos
, , interp
NAJWAŻNIEJSZE ważny adj:sg:nom:n:sup
JEST być fin:sg:ter:imperf
DLA dla prep:gen
MNIE ja ppron12:sg:gen:m1:pri:akc
TO to subst:sg:nom:n
, , interp
BY by comp
SKONCENTROWAĆ skoncentrować inf:perf
SIĘ się qub
NA na prep:loc
TEKŚCIE tekst subst:sg:loc:m3
SZEKSPIRA Szekspir subst:sg:gen:m1
, , interp
NA na prep:loc
JEGO on ppron3:sg:gen:m1:ter:akc:npraep
SKOŃCZONYCH skończony adj:pl:loc:f:pos
, , interp
DOSKONAŁYCH

bojownicy bojownik subst:pl:nom:m1
o o prep:acc
sprawę sprawa subst:sg:acc:f
politycznego polityczny adj:sg:gen:m3:pos
islamu islam subst:sg:gen:m3
. . interp
I i conj
stąd stąd adv
będą być bedzie:pl:ter:imperf
się się qub
oni on ppron3:pl:nom:m1:ter:akc:npraep
rozjeżdżać rozjeżdżać inf:imperf
na na prep:acc
cały cały adj:sg:acc:m3:pos
świat świat subst:sg:acc:m3
. . interp
Czy czy qub
wywiady wywiad subst:pl:nom:m3
, , interp
jakich jaki adj:pl:gen:m3:pos
udzielił udzielić praet:sg:m1:perf
" " interp
Gazecie gazeta subst:sg:dat:f
Wyborczej wyborczy adj:sg:dat:f:pos
" " interp
Piotr Piotr subst:sg:nom:m1
Dziurowicz Dziurowicz subst:sg:nom:m1
, , interp
były być praet:pl:m3:imperf
dla dla prep:gen
pana pan subst:sg:gen:m1
zaskoczeniem zaskoczenie subst:sg:inst:n
? ? interp
Początkowo początkowo adv:pos
, , interp
gdy gdy adv
zaczęło zacząć praet:sg:n:perf
się się qub
tyle tyle adv
mówić mówić inf:imperf
o o prep:loc
oszustwach oszustwo subst:pl:loc:n
, , interp
sądził sądzić praet:sg:m

dr doktor brev:npun
Dollar Dollar subst:sg:nom:m1
- - interp
kudłaty kudłaty adj:sg:nom:m2:pos
psiak psiak subst:sg:nom:m2
rasy rasa subst:sg:gen:f
golden golden subst:sg:nom:m2
retriever retriever subst:sg:nom:m2
, , interp
który który adj:sg:nom:m2:pos
całe cały adj:sg:acc:n:pos
życie życie subst:sg:acc:n
spędził spędzić praet:sg:m2:perf
w w prep:loc:nwok
instytucjach instytucja subst:pl:loc:f
takich taki adj:pl:loc:f:pos
jak jak prep:nom
ta ten adj:sg:nom:f:pos
- - interp
przystaje przystawać fin:sg:ter:imperf
i i conj
podaje podawać fin:sg:ter:imperf
łapę łapa subst:sg:acc:f
. . interp
Dziewczyny dziewczyna subst:pl:nom:f
są być fin:pl:ter:imperf
zachwycone zachwycić ppas:pl:nom:f:perf:aff
. . interp
Podobnie podobnie adv:pos
jak jak adv:pos
chorzy chory adj:pl:nom:m1:pos
na na prep:acc
raka rak subst:sg:acc:m2
pacjenci pacjent subst:pl:nom:m1
wyczekujący wyczekiwać pact:pl:nom:m1:imperf:aff
każdej każdy adj:sg:gen:f:pos
wizyty wizyta subst:sg:gen:f
swego swój adj:sg:gen:m1:pos
ulu

Zapowiedział zapowiedzieć praet:sg:m1:perf
, , interp
że że comp
wybory wybory subst:pl:nom:n
parlamentarne parlamentarny adj:pl:nom:n:pos
odbędą odbyć fin:pl:ter:perf
się się qub
23 23 adj:sg:gen:m3:pos
września wrzesień subst:sg:gen:m3
. . interp
Politycy polityk subst:pl:nom:m1
wszystkich wszystek adj:pl:gen:n:pos
ugrupowań ugrupowanie subst:pl:gen:n
zasadniczo zasadniczo adv:pos
pozytywnie pozytywnie adv:pos
ocenili ocenić praet:pl:m1:perf
decyzję decyzja subst:sg:acc:f
Kwaśniewskiego Kwaśniewski subst:sg:gen:m1
. . interp
Bardziej bardzo adv:com
krytycznie krytycznie adv:pos
oceniają oceniać fin:pl:ter:imperf
wyznaczony wyznaczyć ppas:sg:acc:m3:perf:aff
przez przez prep:acc:nwok
prezydenta prezydent subst:sg:acc:m1
termin termin subst:sg:acc:m3
wyborów wybory subst:pl:gen:n
. . interp
Nowa nowy adj:sg:nom:f:pos
ordynacja ordynacja subst:sg:nom:f
wprowadza wprowadzać fin:sg:ter:imperf
zmodyfikowaną zmodyfikować ppas:sg:acc:f:perf:aff
metodę metoda subst:sg:acc:f
St St brev:pun
. . 

KeyboardInterrupt: 